# 00 - Create Parquet file used in subsequent samples

This notebook will create the parquet file used in subsequent samples

In [3]:
val scope = "key-vault-secrets"

val storageAccount = "dmstore2";
val storageKey = dbutils.secrets.get(scope, "dmstore2-2");

val parquetLocation = s"wasbs://tpch@$storageAccount.blob.core.windows.net/10GB/parquet/lineitem"

scope: String = key-vault-secrets
storageAccount: String = dmstore2
storageKey: String = [REDACTED]
parquetLocation: String = wasbs://tpch@dmstore2.blob.core.windows.net/10GB/parquet/lineitem

Configure Spark to access Azure Blob Store

In [5]:
spark.conf.set(s"fs.azure.account.key.$storageAccount.blob.core.windows.net", storageKey);

In [7]:
import org.apache.spark.sql.types._

val li1 = spark
  .read
  .format("csv")
  .option("sep", "|")
  .schema("""
    L_ORDERKEY INTEGER,
    L_PARTKEY INTEGER,
    L_SUPPKEY INTEGER,
    L_LINENUMBER INTEGER,
    L_QUANTITY DECIMAL(15,2),
    L_EXTENDEDPRICE DECIMAL(15,2),
    L_DISCOUNT DECIMAL(15,2),
    L_TAX DECIMAL(15,2),
    L_RETURNFLAG CHAR(1),
    L_LINESTATUS CHAR(1),
    L_SHIPDATE DATE,
    L_COMMITDATE DATE,
    L_RECEIPTDATE DATE,
    L_SHIPINSTRUCT CHAR(25),
    L_SHIPMODE CHAR(10),
    L_COMMENT VARCHAR(44)
   """)
  .load(s"wasbs://tpch@$storageAccount.blob.core.windows.net/10GB/lineitem.tbl")
;

import org.apache.spark.sql.types._
li1: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 14 more fields]

Create a temporary view to make it easier to manipulate schema and data

In [9]:
li1.createOrReplaceTempView("LINEITEM")

Add a new column that will be used for partitioning

In [11]:
var li2 = spark.sql("SELECT *, YEAR(L_COMMITDATE) * 100 + MONTH(L_COMMITDATE) AS L_PARTITION_KEY FROM LINEITEM")

li2: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

Repartition data using the newly created column

In [13]:
val li3 = li2.repartition($"L_PARTITION_KEY")

li3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

Save dataframe into parquet format, making sure parquet will be saved using the same partitioning logic used for the dataframe

In [15]:
li3.write.mode("overwrite").partitionBy("L_PARTITION_KEY").parquet(parquetLocation)

As a test read back the parquet files

In [17]:
val li4 = spark.read.parquet(parquetLocation)
li4.rdd.getNumPartitions

li4: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]
res6: Int = 20

Peek at first 10 partitions

In [19]:
display(li4.groupBy($"L_PARTITION_KEY").count().orderBy($"L_PARTITION_KEY").limit(10))

L_PARTITION_KEY,count
199201,412
199202,190252
199203,582150
199204,748645
199205,770266
199206,746006
199207,772006
199208,770213
199209,748997
199210,771256
